# Copyright 2026 Cognite AS

## Import the Libraries and Modules

In [ ]:
import sys
from pathlib import Path

utils = str(Path("../utils").resolve())
if utils not in sys.path:
    sys.path.append(utils)

from cognite_auth import interactive_client

## Create the Cognite Client

In [ ]:
c = interactive_client()

## Create various resource types

Add your prefix to make sure that the data we create are unique.
You can for example use your name as a prefix

In [ ]:
prefix = "PREFIX" #REMEBER TO CHANGE THIS 

### CogniteAssets 

##### Here we are creating 2 seperate CogniteAssets

In [ ]:
from cognite.client.data_classes.data_modeling import ViewId, NodeOrEdgeData, NodeApply

COGNITE_ASSET_VIEW = ViewId("cdf_cdm", "CogniteAsset", "v1")

assets = [
    NodeApply(
        space="playground",
        external_id="asset1_" + prefix,
        sources=[
            NodeOrEdgeData(
                source=COGNITE_ASSET_VIEW,
                properties={
                    "name": "asset1",
                },
            )
        ],
    ),
    NodeApply(
        space="playground",
        external_id="asset2_" + prefix,
        sources=[
            NodeOrEdgeData(
                source=COGNITE_ASSET_VIEW,
                properties={
                    "name": "asset2",
                },
            )
        ],
    ),
]

client.data_modeling.instances.apply(nodes=assets)

##### In this learning envirnoment, everyone adds the CogniteAssets with the same name into the same environment. If we wanted to search for the CogniteAssets just created, we could have filtered on name or similar, but since we will have a lot of duplicated assets with the same name (bit not same exteranl id), we can run the cell below to search for our CogniteAssets with external id that ends with the induvidual prefix

In [ ]:
from cognite.client import CogniteClient
from cognite.client.data_classes.data_modeling import ViewId
from cognite.client.data_classes.filters import Prefix


asset_view = ViewId("cdf_cdm", "CogniteAsset", "v1")

# externalId is a base node property → use ("node", "externalId") in filters
flt = Prefix(("node", "externalId"), "")  # empty prefix = no restriction

assets = client.data_modeling.instances.list(
    sources=[asset_view],
    filter=flt,
    limit=None,
)

suffix = prefix
matching = [a for a in assets if a.external_id.endswith(suffix)]

matching

### CogniteAssets Hierachy

##### CogniteAssets can also be organized into a hierarchy. Lets look at how we can create a root node, and to children to that node. 

In [ ]:
root = NodeApply(
    space="playground",
    external_id=prefix + "_root",
    sources=[
        NodeOrEdgeData(
            source=COGNITE_ASSET_VIEW,
            properties={
                "name": prefix + " Root",
            },
        )
    ],
)

child1 = NodeApply(
    space="playground",
    external_id=prefix + "_child1",
    sources=[
        NodeOrEdgeData(
            source=COGNITE_ASSET_VIEW,
            properties={
                "name": "child1",
                "parent": {"space": "playground", "externalId": prefix + "_root"},
            },
        )
    ],
)

child2 = NodeApply(
    space="playground",
    external_id=prefix + "_child2",
    sources=[
        NodeOrEdgeData(
            source=COGNITE_ASSET_VIEW,
            properties={
                "name": "child2",
                "parent": {"space": "playground", "externalId": prefix + "_root"},
            },
        )
    ],
)

child_of_child1 = NodeApply(
    space="playground",
    external_id=prefix + "_child_of_child1",
    sources=[
        NodeOrEdgeData(
            source=COGNITE_ASSET_VIEW,
            properties={
                "name": "child_of_child1",
                "parent": {"space": "playground", "externalId": prefix + "_child1"},
            },
        )
    ],
)

res = client.data_modeling.instances.apply(nodes=[root, child1, child2, child_of_child1])
print(res)

#### Lets look at the CogniteAsset Hierarchy that was created

In [ ]:
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteAsset

# 1) Get the root CogniteAsset node
root = client.data_modeling.instances.retrieve_nodes(
    NodeId("playground", prefix + "_root"),  # space + externalId of the root asset
    node_cls=CogniteAsset,
)

# 2) List all assets in the subtree (root + children)
subtree = client.data_modeling.instances.list(
    instance_type=CogniteAsset,
    filter=Prefix(
        property=["cdf_cdm", "CogniteAsset/v1", "path"],
        value=root.path,
    ),
    limit=None,
)


subtree

##### Or you can go into the CDF user interface (fusion.cognite.com, org=`cognite-learn`, `ds-basics project`) and look at it there. If you do, go to Industrial tools worksapce --> Search: in the filtering you can apply Space=playground. Also changing to "Tree view" might make it easier for you to see and navigate the hierachy created.

### CogniteTimeseries

In [ ]:
from cognite.client.data_classes.data_modeling import DirectRelationReference, NodeId
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteTimeSeriesApply

# Create time series node in CDM
ts = CogniteTimeSeriesApply(
    space="playground",
    external_id="timeseries_" + prefix,
    name="timeseries_" + prefix,
    time_series_type="numeric",
    is_step=False,
    # Optionally link to an asset:
    assets=[DirectRelationReference("playground", "asset1_" + prefix)],
)
client.data_modeling.instances.apply(ts)

# Insert datapoints (same as before, but use instance_id instead of external_id)
from datetime import datetime

datapoints = [(datetime(2026, 1, 1), 1000), (datetime(2026, 1, 2), 2000)]
client.time_series.data.insert(
    instance_id=NodeId("playground", "timeseries_" + prefix),
    datapoints=datapoints,
)

# Retrieve datapoints
client.time_series.data.retrieve(
    instance_id=NodeId("playground", "timeseries_" + prefix),
    start=datetime(2025, 1, 1),
    end=datetime(2027, 1, 1),
)

## Updates in core data model
Instead of client.assets.update(...) etclient., you update nodes via instances.apply again (re‑applying with changed properties), or use typed *Apply classes. The knowledge sources don’t give a full example of partial updates for CogniteAsset, so I can’t show a fully documented pattern here; the general rule is:

Re‑send the node with updated properties via `instances.apply(...)`.


##### Lets change the names of the two first CogniteAssets created, and add a description. Please change the description string to something custom. This onøy makes it easier to search for your CogniteAssets

In [ ]:
from cognite.client.data_classes.data_modeling import ViewId, NodeOrEdgeData, NodeApply

assets = [
    NodeApply(
        space="playground",
        external_id="asset1_" + prefix,
        sources=[
            NodeOrEdgeData(
                source=COGNITE_ASSET_VIEW,
                properties={
                    "name": prefix + "_asset1",
                    "description": "Change the description to more easily recognize your own CogniteAsset", #CHANGE THIS
                },
            )
        ],
    ),
    NodeApply(
        space="playground",
        external_id="asset2_" + prefix,
        sources=[
            NodeOrEdgeData(
                source=COGNITE_ASSET_VIEW,
                properties={
                    "name": prefix + "_asset2",
                    "description": "Change the description to more easily recognize your own CogniteAsset", #CHANGE THIS
                },
            )
        ],
    ),
]

client.data_modeling.instances.apply(nodes=assets)

##### Now, go to fusion.cognite.com, Industrial tools --> Search, and see if you can find your updated CogniteAssets. 